# VAE

## Evidence Lower BOund (ELBO)

We want to max the likelihood p(x) of the data. But p(x) is intractable. We need a lower bound, i.e., ELBO that we can optimize.

# Variational Autoencoder (VAE) - Formulation mathématique

## Distribution Prior
Le prior sur la variable latente $z$ est une distribution normale standard:
$p(z) = \mathcal{N}(0, I)$

## Encoder (Approximate Posterior)
L'encodeur produit une distribution gaussienne avec moyenne et variance pour chaque entrée $x$:
$q_\theta(z|x) = \mathcal{N}(z; \mu_\theta(x), \Sigma_\theta(x))$

où:
- $\mu_\theta(x)$ est le vecteur moyenne (2D)
- $\Sigma_\theta(x)$ est la matrice de covariance diagonale (2D)

## Decoder
Le décodeur produit également une distribution gaussienne:
$p(x|z) = \mathcal{N}(x; \mu_\phi(z), \Sigma_\phi(z))$

où:
- $\mu_\phi(z)$ est le vecteur moyenne (dimension d'entrée)
- $\Sigma_\phi(z)$ est la matrice de covariance diagonale (dimension d'entrée)

## Reparametrization Trick
Pour permettre la rétropropagation à travers l'échantillonnage:
$z = \mu_\theta(x) + \Sigma_\theta(x)^{1/2} \odot \epsilon, \text{ où } \epsilon \sim \mathcal{N}(0, I)$

## Evidence Lower BOund (ELBO)
L'objectif est de maximiser:
$\text{ELBO} = \mathbb{E}_{q(z|x)}[\log p(x|z)] - \text{KL}(q_\theta(z|x) \| p(z))$

Cette expression se décompose en:

1. **Terme de reconstruction**:
$\mathbb{E}_{q(z|x)}[\log p(x|z)]$

Pour une distribution gaussienne:
$-\log p(x|z) = \frac{1}{2} (\log(2\pi) + \log(\Sigma_\phi(z)) + \frac{(x - \mu_\phi(z))^2}{\Sigma_\phi(z)})$

2. **Terme KL**:

Pour des distributions gaussiennes, il a une forme fermée:
$\text{KL} = \frac{1}{2} \sum (\mu_\theta(x)^2 + \Sigma_\theta(x) - \log(\Sigma_\theta(x)) - 1)$

## Loss totale
On minimise la negative ELBO:
$\mathcal{L} = -\mathbb{E}_{q(z|x)}[\log p(x|z)] + \text{KL}(q_\theta(z|x) \| p(z))$

## Dimensions
- $x \in \mathbb{R}^D$ (dimension d'entrée)
- $z \in \mathbb{R}^2$ (dimension latente = 2)
- $\mu_\theta(x) \in \mathbb{R}^2$
- $\Sigma_\theta(x) \in \mathbb{R}^2$ (covariance diagonale)
- $\mu_\phi(z) \in \mathbb{R}^D$
- $\Sigma_\phi(z) \in \mathbb{R}^D$ (covariance diagonale)

---

# Reconstruction Loss pour une distribution Gaussienne

## Formule générale
Pour une distribution gaussienne, la log-vraisemblance négative (reconstruction loss) est:

$-\log p(x|z) = \frac{1}{2} (\log(2\pi) + \log(\Sigma_\phi(z)) + \frac{(x - \mu_\phi(z))^2}{\Sigma_\phi(z)})$

## Décomposition du terme
1. **Terme constant**: $\frac{1}{2}\log(2\pi)$
  - Ce terme est constant et n'affecte pas l'optimisation
  - Il peut être ignoré pendant l'entraînement

2. **Terme de variance**: $\frac{1}{2}\log(\Sigma_\phi(z))$
  - Pénalise les grandes variances
  - Encourage le modèle à être confiant dans ses prédictions
  - Pour une variance $\sigma^2$ proche de 0, $\log(\sigma^2)$ tend vers $-\infty$
  - Pour une variance $\sigma^2$ grande, $\log(\sigma^2)$ devient positif

3. **Terme d'erreur quadratique normalisée**: $\frac{1}{2}\frac{(x - \mu_\phi(z))^2}{\Sigma_\phi(z)}$
  - Mesure l'erreur de reconstruction
  - Pondérée par l'inverse de la variance
  - Si $\Sigma_\phi(z)$ est petit:
    - Grande pénalité pour les erreurs
    - Le modèle doit être très précis
  - Si $\Sigma_\phi(z)$ est grand:
    - Faible pénalité pour les erreurs
    - Le modèle peut être moins précis

## En pratique
- Pour chaque dimension $i$:
$-\log p(x_i|z) = \frac{1}{2} (\log(2\pi) + \log(\sigma^2_i) + \frac{(x_i - \mu_i)^2}{\sigma^2_i})$

- La loss totale est la somme sur toutes les dimensions:
$-\log p(x|z) = \sum_i -\log p(x_i|z)$

## Trade-off Variance-Précision
Il y a un équilibre à trouver:
- Une variance trop faible force une reconstruction très précise mais risque d'overfitting
- Une variance trop élevée permet trop d'imprécision
- Le modèle doit apprendre à moduler la variance selon la difficulté de reconstruction

# Dérivation de la Reconstruction Loss Gaussienne

## 1. Rappel densité Gaussienne
Pour une gaussienne univariée:
$p(x|z) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp(-\frac{(x-\mu)^2}{2\sigma^2})$

où $\mu = \mu_\phi(z)$ et $\sigma^2 = \Sigma_\phi(z)$

## 2. Application du log négatif
$-\log p(x|z) = -\log(\frac{1}{\sqrt{2\pi\sigma^2}} \exp(-\frac{(x-\mu)^2}{2\sigma^2}))$

## 3. Propriétés des logs
$-\log p(x|z) = -[-\frac{1}{2}\log(2\pi\sigma^2) - \frac{(x-\mu)^2}{2\sigma^2}]$

## 4. Distributivité du négatif
$-\log p(x|z) = \frac{1}{2}\log(2\pi\sigma^2) + \frac{(x-\mu)^2}{2\sigma^2}$

## 5. Propriété du log
$\log(2\pi\sigma^2) = \log(2\pi) + \log(\sigma^2)$

## 6. Formule finale
$-\log p(x|z) = \frac{1}{2}(\log(2\pi) + \log(\sigma^2) + \frac{(x-\mu)^2}{\sigma^2})$